### Shared Word File Process

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [4]:
df_shared_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx")
df_shared_file

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,salam,1768.0,bir şey,859944.0,bir şey yok,113165,bir şey var mı,41773.0,komik bir şey mi var,901.0,ne var,62532.0
1,depresyon,3215.0,değil mi,585879.0,bir şey var,110455,bir şey daha var,12185.0,bir şey yok değil mi,893.0,ben de,59972.0
2,meteoroloji,1212.0,ben de,377765.0,bu da ne,89463,bir şey mi var,11834.0,o kadar çok şey var,790.0,değil mi,58386.0
3,arbitraj,39.0,var mı,281902.0,bir şey değil,68498,ne var ne yok,9276.0,bir şey yok bir şey,490.0,bu da ne,50098.0
4,poligon,158.0,ne kadar,237876.0,bir şey mi,64933,bir şey değil mi,5080.0,ama bir şey daha var,393.0,ben mi,33652.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13141,NaN,NaN,NaN,NaN,şeker mi var,10,NaN,NaN,NaN,NaN,NaN,NaN
13142,NaN,NaN,NaN,NaN,bir dj var,10,NaN,NaN,NaN,NaN,NaN,NaN
13143,NaN,NaN,NaN,NaN,kantin ve kulüp,10,NaN,NaN,NaN,NaN,NaN,NaN
13144,NaN,NaN,NaN,NaN,şeker için bana,10,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [25]:
def word_in_wordgroup(df, list_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            #word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(10)  # Option
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(100) 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{list_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [9]:
word_count_result(df_shared_file,["threegram"])

,word,word_count
0,bir,5317
1,bu,2072
2,ve,1695
3,de,1415
4,için,1241
...,...,...
1071,download,1
1072,paralel,1
1073,paragraf,1
1074,panel,1


In [30]:
df_shared_file["twogram"].nunique()

3107

In [26]:
df_two_in_three = word_in_wordgroup(df_shared_file, "twogram", "threegram")
df_two_in_three

,twogram,threegram
0,bir şey,bir şey yok
1,bir şey,bir şey var
2,bir şey,bir şey değil
3,bir şey,bir şey mi
4,bir şey,bir şey daha
...,...,...
14038,karakter mi,bir karakter mi
14039,alkolik değil,bir alkolik değil
14040,kariyer değil,bir kariyer değil
14041,şey general,bir şey general


In [29]:
df_two_in_three["twogram"].nunique()

1937

In [10]:
df_hybrids = pd.read_excel("Turkish English manual selected 2 gram hybrids.xlsx")
df_hybrids

,word,twogram_1,twogram_2,twogram_3,twogram_4
0,bir,bir doktor,bir polis,bir dolar,bir telefon
1,bu,bu doktor,bu komik,bu telefon,bu bebek
2,ne,ne şans,mesaj ne,plan ne,ne sürpriz
3,ve,ve müzik,ve şeker,ve patates,ve çikolata
4,için,kontrol için,bebek için,dans için,film için
5,mi,komik mi,normal mi,problem mi,profesyonel mi
6,o,o ajan,o müzik,o park,o film
7,ben,ben senatör,ben ajan,ben general,ben profesör
8,de,patates de,şef de,profesör de,prenses de
9,çok,çok komik,çok normal,çok pardon,çok süper


In [13]:
word_count_result(df_hybrids, ["twogram_1","twogram_2","twogram_3","twogram_4"])

,word,word_count
0,bebek,6
1,komik,6
2,normal,5
3,sen,4
4,onu,4
...,...,...
89,okula,1
90,park,1
91,parka,1
92,pizza,1


In [11]:
df_hybrids_2 = pd.read_excel("Turkish English manual selected 2 gram hybrids 2.xlsx")
df_hybrids_2

,word,twogram_1,twogram_2,twogram_3,twogram_4
0,bir,bir doktor,bir polis,bir dolar,bir telefon
1,bu,bu doktor,bu komik,bu telefon,bu bebek
2,ne,ne şans,mesaj ne,plan ne,ne sürpriz
3,ve,ve müzik,ve şeker,ve patates,ve çikolata
4,için,kontrol için,bebek için,dans için,film için
5,mi,komik mi,normal mi,problem mi,profesyonel mi
6,o,o ajan,o müzik,o park,o film
7,ben,ben senatör,ben ajan,ben general,ben profesör
8,de,patates de,şef de,profesör de,prenses de
9,çok,çok komik,çok normal,çok pardon,çok süper


In [14]:
word_count_result(df_hybrids_2, ["twogram_1","twogram_2","twogram_3","twogram_4"])

,word,word_count
0,bebek,6
1,komik,6
2,normal,5
3,sen,4
4,onu,4
...,...,...
89,okula,1
90,park,1
91,parka,1
92,pizza,1


In [19]:
df_concat_1_2 = pd.concat([df_hybrids, df_hybrids_2], axis=0)
#df_concat_1_2.drop_duplicates(inplace=True)
df_concat_1_2.reset_index(drop=True, inplace=True)
df_concat_1_2


,word,twogram_1,twogram_2,twogram_3,twogram_4
0,bir,bir doktor,bir polis,bir dolar,bir telefon
1,bu,bu doktor,bu komik,bu telefon,bu bebek
2,ne,ne şans,mesaj ne,plan ne,ne sürpriz
3,ve,ve müzik,ve şeker,ve patates,ve çikolata
4,için,kontrol için,bebek için,dans için,film için
5,mi,komik mi,normal mi,problem mi,profesyonel mi
6,o,o ajan,o müzik,o park,o film
7,ben,ben senatör,ben ajan,ben general,ben profesör
8,de,patates de,şef de,profesör de,prenses de
9,çok,çok komik,çok normal,çok pardon,çok süper


In [20]:
word_count_result(df_hybrids_2, ["twogram_1","twogram_2","twogram_3","twogram_4"])

,word,word_count
0,bebek,6
1,komik,6
2,normal,5
3,sen,4
4,onu,4
...,...,...
89,okula,1
90,park,1
91,parka,1
92,pizza,1
